In [ ]:
# Install libraries
!pip install requests
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
# Import libraries
import requests
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# Download data
tickers = ['AAPL', 'IVV', 'ACWI', 'AMZN', 'LEMB', 'IAU', "WMT", "DAVA"]

start = datetime.datetime(2016,1,1)
end = datetime.datetime(2020,1,1)

data = yf.download(tickers, start=start, end=end)
data

In [ ]:
# Select Close Prices
dataPrices = data["Close"]
dataPrices

In [ ]:
# Prices Plot
dataPrices.plot(figsize = (20,10))

In [ ]:
# Logarithm Returs
dataReturns = np.log(dataPrices).diff()
dataReturns = dataReturns.iloc[1:,:].dropna()
dataReturns

In [ ]:
# Return Plots
dataReturns.plot(figsize = (20,8))
dataReturns.plot(kind = "kde", figsize = (20,8))

In [ ]:
# Annualized Expected Returns
dataExpectedAnnualizedReturns = ((dataReturns.mean() + 1)**(252)-1).to_numpy()

dataExpectedAnnualizedReturnsDF = {}
for counter, symbol in enumerate(dataReturns.columns.tolist()):
    dataExpectedAnnualizedReturnsDF[symbol+' Exp. Annu. Ret.'] = round(dataExpectedAnnualizedReturns[counter],3)

dataExpectedAnnualizedReturnsDF

In [ ]:
# Annualized Variance and Covariance
dataVarCovar = dataReturns.cov()*252
dataVarCovar

In [ ]:
# Correlation Matrix
dataCorr = dataReturns.corr()
dataCorr

In [ ]:
# Annualized Volatility
np.sqrt(dataVarCovar)

### To remember

$$
A^{-1}=\frac{1}{|A|} \cdot \operatorname{Adj} A
$$

In [ ]:
# Determinant of Var-Covar Matrix
np.linalg.det(dataVarCovar)

In [ ]:
# Annualized Volatility per Asset
dataAnnualizedVolatilityPerAsset = ((dataReturns.std() )* np.sqrt(252)).to_numpy()

dataAnnualizedVolatilityPerAssetDF = {}
for counter, symbol in enumerate(dataReturns.columns.tolist()):
    dataAnnualizedVolatilityPerAssetDF[symbol+' Annu. Vol.'] = round(dataAnnualizedVolatilityPerAsset[counter],3)

dataAnnualizedVolatilityPerAssetDF

In [ ]:
# Individual Sharpe Ratio
dataSharpeRatioPerAssetDF = {}
rf = 0.01

for symbol in dataReturns.columns.tolist():
    dataSharpeRatioPerAssetDF[symbol+' Sharpe Ratio'] = round( (dataExpectedAnnualizedReturnsDF[symbol+' Exp. Annu. Ret.'] - rf) / dataAnnualizedVolatilityPerAssetDF[symbol+' Annu. Vol.'], 3)

dataSharpeRatioPerAssetDF

In [ ]:
# Function to determine equally weighted portfolios
def pesosPortafolio(dataframe):
    array = []
    for i in dataframe.columns:
        array.append(1/len(dataframe.columns))
    arrayFinal = np.array(array)
    return arrayFinal

pesos = pesosPortafolio(dataReturns)
pesos

pesosEquallyWeightedDF = {}
for counter, symbol in enumerate(dataReturns.columns.tolist()):
    pesosEquallyWeightedDF[symbol+' weight'] = round(pesos[counter],3)

pesosEquallyWeightedDF

# Characterisctics of Equally Weighted Portfolio

In [ ]:
#Varianza del Portafolio equally weighted
varianza_portafolio = pesos.T @ dataVarCovar @ pesos
retorno_esperado_port = pesos.T @ ((dataReturns.mean() + 1)**(252)-1).to_numpy()
print("##########################################################################")
print("################    Analisis portafolio equally weighted     #############")
print("##########################################################################")
print("La varianza anualizada del portafolio es:" + " " + str(round(varianza_portafolio*100,1))+"%")
print("La volatilidad anualizada del portafolio es:" + " " + str(round(np.sqrt(varianza_portafolio)*100,1))+"%")
print("El retorno esperado del portafolio es:" + " " + str(round(retorno_esperado_port*100,1))+"%")
print("El ratio de sharpe portafolio es:" + " " + str(round((retorno_esperado_port - rf) / np.sqrt(varianza_portafolio),1)))

# N Portfolios Simulations

In [ ]:
# Simulation of N portfolios changing the weights of each asset

p_ret = [] # Define an empty array for portfolio returns
p_vol = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights

num_assets = len(dataReturns.columns)
num_portfolios = 100000
np.random.seed(1234)

for portfolio in range(num_portfolios):
    weights = (np.random.random(num_assets))
    #weights = (np.random.dirichlet([1] * num_assets) * 3) - 0.3
    weights = weights/np.sum(weights)
    p_weights.append(weights)
    returns = weights.T @ dataExpectedAnnualizedReturns # Returns are the product of individual expected returns of asset and its
                                      # weights
    p_ret.append(returns)
    var = weights.T @ dataVarCovar @ weights# Portfolio Variance
    sd = np.sqrt(var) # Daily standard deviation
    ann_sd = sd # Annual standard deviation = volatility
    p_vol.append(ann_sd)


data = {'Returns':p_ret, 'Volatility':p_vol}

for counter, symbol in enumerate(dataReturns.columns.tolist()):

    weightsPerAsset = []

    for w in p_weights:
        weightsPerAsset.append(w[counter])

    data[symbol+' weight'] = weightsPerAsset


portfolios  = pd.DataFrame(data)
portfolios.head() # Dataframe of the 1000 portfolios created

In [ ]:
# Plot efficient frontier
portfolios.plot.scatter(x='Volatility', y='Returns', marker='o', s=10, alpha=0.3, grid=True, figsize=[10,10])

In [ ]:
# We find the simulated portfolio with min variance / volatility
min_vol_port = portfolios.iloc[portfolios['Volatility'].idxmin()] # idxmin() gives us the minimum value in the column specified.
min_vol_port

# Minimun Variance - Risk Porfolio (Discrete Solution)

In [ ]:
pesosMinVariance = min_vol_port.to_numpy()[2:]

pesosMinVarianceDF = {}
for counter, symbol in enumerate(dataReturns.columns.tolist()):
    pesosMinVarianceDF[symbol+' weight'] = round(pesosMinVariance[counter],3)

pesosMinVarianceDF

In [ ]:
#Varianza del Portafolio
varianza_portafolioMinVar = pesosMinVariance.T @ dataVarCovar @ pesosMinVariance
retorno_esperado_portMinVar = pesosMinVariance.T @ ((dataReturns.mean() + 1)**(252)-1).to_numpy()
print("##########################################################################")
print("################      Analisis portafolio min variance       #############")
print("##########################################################################")
print("La varianza anualizada del portafolio es:" + " " + str(round(varianza_portafolioMinVar*100,1))+"%")
print("La volatilidad anualizada del portafolio es:" + " " + str(round(np.sqrt(varianza_portafolioMinVar)*100,1))+"%")
print("El retorno esperado del portafolio es:" + " " + str(round(retorno_esperado_portMinVar*100,1))+"%")
print("El ratio de sharpe portafolio es:" + " " + str(round((retorno_esperado_portMinVar - rf) / np.sqrt(varianza_portafolioMinVar),1)))

In [ ]:
# Plotting the minimum volatility portfolio
plt.subplots(figsize=[10,10])
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.xlabel("Volatility")
plt.ylabel("Returns")

# Minimun Variance - Risk Portfolio (Closed Solution)

$$w = \frac{\Sigma^{-1} 1}{\mathbf{1}^{\prime} \Sigma^{-1} 1}$$

In [ ]:
pesosMinVarClosedSolution = np.linalg.inv(dataVarCovar) @ (np.ones(len(dataExpectedAnnualizedReturns))) / (np.ones(len(dataExpectedAnnualizedReturns)).T @ np.linalg.inv(dataVarCovar) @ (np.ones(len(dataExpectedAnnualizedReturns))))

pesosMinVarClosedSolutionDF = {}
for counter, symbol in enumerate(dataReturns.columns.tolist()):
    pesosMinVarClosedSolutionDF[symbol+' weight'] = round(pesosMinVarClosedSolution[counter],3)

pesosMinVarClosedSolutionDF

In [ ]:
retorno_esperado_portMinVarCS = pesosMinVarClosedSolution.T @ dataExpectedAnnualizedReturns # retorno_esperado_portMinVarCS are the product of individual expected returns_sharpe of asset and its pesosMinVarClosedSolution
varianza_portafolioMinVarCS = pesosMinVarClosedSolution.T @ dataVarCovar @ pesosMinVarClosedSolution# Portfolio Variance

print("##########################################################################")
print("#########    Analisis portafolio min Varianza Closed Solution     ########")
print("##########################################################################")
print("La varianza anualizada del portafolio es:" + " " + str(round(varianza_portafolioMinVarCS*100,1))+"%")
print("La volatilidad anualizada del portafolio es:" + " " + str(round(np.sqrt(varianza_portafolioMinVarCS)*100,1))+"%")
print("El retorno esperado del portafolio es:" + " " + str(round(retorno_esperado_portMinVarCS*100,1))+"%")
print("El ratio de sharpe portafolio es:" + " " + str(round((retorno_esperado_portMinVarCS - rf) / np.sqrt(varianza_portafolioMinVarCS),1)))

In [ ]:
# Plotting optimal portfolio
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMinVarCS), retorno_esperado_portMinVarCS, color='r', marker='>', s=500)
plt.xlabel("Volatility")
plt.ylabel("Returns")

# Maximun Sharpe Ratio Portfolio (Discrete Solution)


In [ ]:
# Finding the optimal portfolio
portfolios["SharpeRatio"] = (portfolios['Returns']-rf) / portfolios['Volatility']
optimal_risky_port = portfolios.iloc[portfolios["SharpeRatio"].idxmax()]
optimal_risky_port

In [ ]:
# Plotting optimal portfolio
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMinVarCS), retorno_esperado_portMinVarCS, color='r', marker='>', s=500)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=500)
plt.xlabel("Volatility")
plt.ylabel("Returns")

In [ ]:
pesosMaxRS = optimal_risky_port.to_numpy()[2:len(optimal_risky_port)-1]

pesosMaxRSDF = {}
for counter, symbol in enumerate(dataReturns.columns.tolist()):
    pesosMaxRSDF[symbol+' weight'] = round(pesosMaxRS[counter],3)

pesosMaxRSDF

In [ ]:
#Varianza del Portafolio
varianza_portafolioMaxRS = pesosMaxRS.T @ dataVarCovar @ pesosMaxRS
retorno_esperado_portMaxRS = pesosMaxRS.T @ ((dataReturns.mean() + 1)**(252)-1).to_numpy()
print("##########################################################################")
print("################    Analisis portafolio max Ratio Sharpe     #############")
print("##########################################################################")
print("La varianza anualizada del portafolio es:" + " " + str(round(varianza_portafolioMaxRS*100,1))+"%")
print("La volatilidad anualizada del portafolio es:" + " " + str(round(np.sqrt(varianza_portafolioMaxRS)*100,1))+"%")
print("El retorno esperado del portafolio es:" + " " + str(round(retorno_esperado_portMaxRS*100,1))+"%")
print("El ratio de sharpe portafolio es:" + " " + str(round((retorno_esperado_portMaxRS - rf) / np.sqrt(varianza_portafolioMaxRS),1)))

# Maximun Sharpe Ratio Portfolio (Closed Solution)

$$
 w = \frac{\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{\bar{R}}-\boldsymbol{r_{f}}\right)}{\mathbf{1}^{\prime} \boldsymbol{\Sigma}^{-1}\left(\boldsymbol{\bar{R}}-\boldsymbol{r_{f}}\right)}
$$

In [ ]:
pesosMaxRSClosedSolution = np.linalg.inv(dataVarCovar) @ (dataExpectedAnnualizedReturns - rf) / (np.ones(len(dataExpectedAnnualizedReturns)).T @ np.linalg.inv(dataVarCovar) @ (dataExpectedAnnualizedReturns - rf))

pesosMaxRSClosedSolutionDF = {}
for counter, symbol in enumerate(dataReturns.columns.tolist()):
    pesosMaxRSClosedSolutionDF[symbol+' weight'] = round(pesosMaxRSClosedSolution[counter],3)

pesosMaxRSClosedSolutionDF

In [ ]:
retorno_esperado_portMaxRSCS = pesosMaxRSClosedSolution.T @ dataExpectedAnnualizedReturns # retorno_esperado_portMaxRSCS are the product of individual expected returns_sharpe of asset and its pesosMaxRSClosedSolution
varianza_portafolioMaxRSCS = pesosMaxRSClosedSolution.T @ dataVarCovar @ pesosMaxRSClosedSolution# Portfolio Variance

print("##########################################################################")
print("#########  Analisis portafolio max Ratio Sharpe Closed Solution   ########")
print("##########################################################################")
print("La varianza anualizada del portafolio es:" + " " + str(round(varianza_portafolioMaxRSCS*100,1))+"%")
print("La volatilidad anualizada del portafolio es:" + " " + str(round(np.sqrt(varianza_portafolioMaxRSCS)*100,1))+"%")
print("El retorno esperado del portafolio es:" + " " + str(round(retorno_esperado_portMaxRSCS*100,1))+"%")
print("El ratio de sharpe portafolio es:" + " " + str(round((retorno_esperado_portMaxRSCS - rf) / np.sqrt(varianza_portafolioMaxRSCS),1)))

In [ ]:
# Plotting optimal portfolio
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMinVarCS), retorno_esperado_portMinVarCS, color='r', marker='>', s=500)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMaxRSCS), retorno_esperado_portMaxRSCS, color='g', marker='>', s=500)
# plt.ylim(0, 0.5)
# plt.xlim(0, 0.5)
plt.xlabel("Volatility")
plt.ylabel("Returns")

# Asset's Weights by Regions in Space $\sigma$ - $\bar{R}$

In [ ]:
# Plotting optimal portfolio
asset = 1
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'], c = portfolios.iloc[:,asset + 1],marker='o', s=10, alpha=0.3, vmin = portfolios.iloc[:,asset + 1].min(), vmax = portfolios.iloc[:,asset + 1].max(), cmap='rainbow')
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMinVarCS), retorno_esperado_portMinVarCS, color='r', marker='>', s=500)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMaxRSCS), retorno_esperado_portMaxRSCS, color='g', marker='>', s=500)
plt.ylim(0, 0.5)
plt.xlim(0, 0.5)
plt.colorbar(label='Weight of ' + portfolios.columns[asset + 1])
plt.xlabel("Volatility")
plt.ylabel("Returns")

In [ ]:
# Plotting optimal portfolio
asset = 2
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'], c = portfolios.iloc[:,asset + 1],marker='o', s=10, alpha=0.3, vmin = portfolios.iloc[:,asset + 1].min(), vmax = portfolios.iloc[:,asset + 1].max(), cmap='rainbow')
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMinVarCS), retorno_esperado_portMinVarCS, color='r', marker='>', s=500)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMaxRSCS), retorno_esperado_portMaxRSCS, color='g', marker='>', s=500)
plt.ylim(0, 0.5)
plt.xlim(0, 0.5)
plt.colorbar(label='Weight of ' + portfolios.columns[asset + 1])
plt.xlabel("Volatility")
plt.ylabel("Returns")

In [ ]:
# Plotting optimal portfolio
asset = 3
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'], c = portfolios.iloc[:,asset + 1],marker='o', s=10, alpha=0.3, vmin = portfolios.iloc[:,asset + 1].min(), vmax = portfolios.iloc[:,asset + 1].max(), cmap='rainbow')
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMinVarCS), retorno_esperado_portMinVarCS, color='r', marker='>', s=500)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMaxRSCS), retorno_esperado_portMaxRSCS, color='g', marker='>', s=500)
plt.ylim(0, 0.5)
plt.xlim(0, 0.5)
plt.colorbar(label='Weight of ' + portfolios.columns[asset + 1])
plt.xlabel("Volatility")
plt.ylabel("Returns")

# How Change Asset's Weights Thought Time?

In [ ]:
lastDateRebalance = dataReturns.index[-252]
listDates = dataReturns[lastDateRebalance:].index.to_list()
listDates = [listDates[x] for x in range(1,len(listDates)-1,5)]

dictHistWeigthsMVDiscrete = {}
dictHistWeigthsMVCS = {}

for dateRebalance in listDates:
    dataReturnsRebalance = dataReturns[:dateRebalance]
    dataVarCovar = dataReturnsRebalance.cov()*252
    dataExpectedAnnualizedReturns = ((dataReturnsRebalance.mean() + 1)**(252)-1).to_numpy()

    # Simulation of N portfolios changing the weights of each asset

    p_ret = [] # Define an empty array for portfolio returns
    p_vol = [] # Define an empty array for portfolio volatility
    p_weights = [] # Define an empty array for asset weights

    num_assets = len(dataReturns.columns)
    num_portfolios = 1000
    np.random.seed(1234)

    for portfolio in range(num_portfolios):
        weights = (np.random.random(num_assets))
        # weights = (np.random.dirichlet([1] * num_assets) * 3) - 0.3
        weights = weights/np.sum(weights)
        p_weights.append(weights)
        returns = weights.T @ dataExpectedAnnualizedReturns # Returns are the product of individual expected returns of asset and its
                                          # weights
        p_ret.append(returns)
        var = weights.T @ dataVarCovar @ weights# Portfolio Variance
        sd = np.sqrt(var) # Daily standard deviation
        ann_sd = sd # Annual standard deviation = volatility
        p_vol.append(ann_sd)


    data = {'Returns':p_ret, 'Volatility':p_vol}

    for counter, symbol in enumerate(dataReturns.columns.tolist()):

        weightsPerAsset = []

        for w in p_weights:
            weightsPerAsset.append(w[counter])

        data[symbol+' weight'] = weightsPerAsset


    portfolios  = pd.DataFrame(data)
    portfolios.head() # Dataframe of the 1000 portfolios created

    min_vol_port = portfolios.iloc[portfolios['Volatility'].idxmin()] # idxmin() gives us the minimum value in the column specified.

    dictHistWeigthsMVDiscrete[dateRebalance] = min_vol_port.to_dict()



    pesosMinVarClosedSolution = np.linalg.inv(dataVarCovar) @ (np.ones(len(dataExpectedAnnualizedReturns))) / (np.ones(len(dataExpectedAnnualizedReturns)).T @ np.linalg.inv(dataVarCovar) @ (np.ones(len(dataExpectedAnnualizedReturns))))

    pesosMinVarClosedSolutionDF = {}
    for counter, symbol in enumerate(dataReturns.columns.tolist()):
        pesosMinVarClosedSolutionDF[symbol+' weight'] = round(pesosMinVarClosedSolution[counter],3)

    dictHistWeigthsMVCS[dateRebalance] = pesosMinVarClosedSolutionDF



In [ ]:
pd.DataFrame(dictHistWeigthsMVDiscrete).T.plot(figsize = (15,10))

In [ ]:
pd.DataFrame(dictHistWeigthsMVCS).T.plot(figsize = (15,10))

In [ ]:
dfHistWeigthsMVDiscrete = pd.DataFrame(dictHistWeigthsMVDiscrete).T
initialWeigths = dfHistWeigthsMVDiscrete.iloc[0,2:]
initialValueInvestment = 100

dateRebalanceInter = dfHistWeigthsMVDiscrete.index.to_list()[0]

dictWeightsRebalanceIntra = {}
dictinitialValueInvestment = {}

for dateRebalanceInter in dfHistWeigthsMVDiscrete.index.to_list():

    dataReturnsDRI = dataReturns.loc[dateRebalanceInter]
    investmentPerAsset = initialWeigths * initialValueInvestment
    newInvestmentPerAsset = investmentPerAsset.to_numpy() * (1 + dataReturnsDRI)
    newWeigths = newInvestmentPerAsset / newInvestmentPerAsset.sum()
    newValueInvestment = newInvestmentPerAsset.sum()

    initialWeigths = newWeigths
    initialValueInvestment = newValueInvestment

    dictWeightsRebalanceIntra[dateRebalanceInter] = initialWeigths
    dictinitialValueInvestment[dateRebalanceInter] = initialValueInvestment


In [ ]:
def rebalanceIntra(initialValueInvestment, initialWeigths, dataReturns, listDates):

    dfHistWeigthsMVDiscrete = pd.DataFrame(dictHistWeigthsMVDiscrete).T
    dateRebalanceInter = dfHistWeigthsMVDiscrete.loc[listDates[0]]

    dictWeightsRebalanceIntra = {}
    dictinitialValueInvestment = {}

    for dateRebalanceInter in listDates:

        dataReturnsDRI = dataReturns.loc[dateRebalanceInter]
        investmentPerAsset = initialWeigths * initialValueInvestment
        newInvestmentPerAsset = investmentPerAsset.to_numpy() * (1 + dataReturnsDRI)
        newWeigths = newInvestmentPerAsset / newInvestmentPerAsset.sum()
        newValueInvestment = newInvestmentPerAsset.sum()

        initialWeigths = newWeigths
        initialValueInvestment = newValueInvestment

        dictWeightsRebalanceIntra[dateRebalanceInter] = initialWeigths
        dictinitialValueInvestment[dateRebalanceInter] = initialValueInvestment

    return([dictWeightsRebalanceIntra, dictinitialValueInvestment])


In [ ]:
lastDateRebalance = dataReturns.index[-252]
listDates = dataReturns[lastDateRebalance:].index.to_list()
listDates = [listDates[x] for x in range(1,len(listDates)-1,5)]

listListDates = []

for index, dateReb in enumerate(listDates[:-1]):
    # listListDates.append(pd.date_range(dateReb, listDates[index]))
    listListDates.append(dataReturns.loc[listDates[index]:listDates[index + 1]].index)

dfHistWeigthsMVDiscrete = pd.DataFrame(dictHistWeigthsMVDiscrete).T

dictInfo = {}

dfValueRebal = pd.DataFrame()
dfWeigthsRebal= pd.DataFrame()

for i, listDates in enumerate(listListDates):
    initialWeigths = dfHistWeigthsMVDiscrete.loc[listDates[0]][2:]
    if i == 0:
        dictInfo[listDates[0]] = rebalanceIntra(100, initialWeigths, dataReturns, listDates)

    else:
        dictInfo[listDates[0]] = rebalanceIntra(dfValueRebal.iloc[-1].item(), initialWeigths, dataReturns, listDates)
    dfValueRebal = pd.concat([dfValueRebal, pd.DataFrame(dictInfo[listDates[0]][1], index = ["Markowitz"]).T])
    dfWeigthsRebal = pd.concat([dfWeigthsRebal, pd.DataFrame(dictInfo[listDates[0]][0]).T])


In [ ]:
dfWeigthsRebal.plot(figsize = (15,10))
plt.title("Weights of assets over time")
plt.ylabel("Time")
plt.ylabel("Weights")

In [ ]:
dfValueRebal
dfValueRebal.plot(figsize = (15,10))
plt.ylabel("Time")
plt.ylabel("Annualized Returns [%]")

In [ ]:
# Calculate the cumulative daily returns
df_cum_daily_returns = (1 + dataReturns.iloc[len(dataReturns) - 252:, :]).cumprod() - 1
df_cum_daily_returns = df_cum_daily_returns
(df_cum_daily_returns*100 + 100).plot(figsize =(15,10))
plt.xlabel("Time")
plt.ylabel("Annualized Return [%]")
plt.title("Buy and Hold Strategy")

In [ ]:
df_cum_daily_return_scaled = (df_cum_daily_returns*100 + 100)
df_cum_daily_return_scaled.join(dfValueRebal, lsuffix="index").plot(figsize = (15,10))

plt.xlabel("Time")
plt.ylabel("Annualized Return [%]")
plt.title("Buy and Hold Strategy VS Markowitz")

# And ML?

### What If Estimate a Robust VarCovar Matrix?

In [ ]:
from sklearn.covariance import LedoitWolf
lw = LedoitWolf()
loglik_lw = lw.fit(dataReturns)
new_var_covar = loglik_lw.covariance_ * 252

In [ ]:
pd.DataFrame(new_var_covar)

In [ ]:
# Determinant of Var-Covar Matrix
np.linalg.det(new_var_covar)

In [ ]:
pesosMaxRSClosedSolutionShrunkCovariance = np.linalg.inv(new_var_covar) @ (dataExpectedAnnualizedReturns - rf) / (np.ones(len(dataExpectedAnnualizedReturns)).T @ np.linalg.inv(new_var_covar) @ (dataExpectedAnnualizedReturns - rf))

pesosMaxRSClosedSolutionDFShrunkCovariance = {}
for counter, symbol in enumerate(dataReturns.columns.tolist()):
    pesosMaxRSClosedSolutionDFShrunkCovariance[symbol+' weight'] = round(pesosMaxRSClosedSolution[counter],3)

pesosMaxRSClosedSolutionDFShrunkCovariance

In [ ]:
retorno_esperado_portMaxRSCSShrunkCovariance = pesosMaxRSClosedSolutionShrunkCovariance.T @ dataExpectedAnnualizedReturns # retorno_esperado_portMaxRSCSShrunkCovariance are the product of individual expected returns_sharpe of asset and its pesosMaxRSClosedSolution
varianza_portafolioMaxRSCSShrunkCovariance = pesosMaxRSClosedSolutionShrunkCovariance.T @ new_var_covar @ pesosMaxRSClosedSolutionShrunkCovariance# Portfolio Variance

print("##########################################################################")
print("#  Analisis portafolio max Ratio Sharpe Closed Solution ShrunkCovariance #")
print("##########################################################################")
print("La varianza anualizada del portafolio es:" + " " + str(round(varianza_portafolioMaxRSCSShrunkCovariance*100,1))+"%")
print("La volatilidad anualizada del portafolio es:" + " " + str(round(np.sqrt(varianza_portafolioMaxRSCSShrunkCovariance)*100,1))+"%")
print("El retorno esperado del portafolio es:" + " " + str(round(retorno_esperado_portMaxRSCSShrunkCovariance*100,1))+"%")
print("El ratio de sharpe portafolio es:" + " " + str(round((retorno_esperado_portMaxRSCSShrunkCovariance - rf) / np.sqrt(varianza_portafolioMaxRSCSShrunkCovariance),1)))

In [ ]:
# Plotting optimal portfolio
plt.subplots(figsize=(10, 10))
plt.scatter(portfolios['Volatility'], portfolios['Returns'],marker='o', s=10, alpha=0.3)
plt.scatter(min_vol_port[1], min_vol_port[0], color='r', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMinVarCS), retorno_esperado_portMinVarCS, color='r', marker='>', s=500)
plt.scatter(optimal_risky_port[1], optimal_risky_port[0], color='g', marker='*', s=500)
plt.scatter(np.sqrt(varianza_portafolioMaxRSCS), retorno_esperado_portMaxRSCS, color='g', marker='>', s=500)
plt.scatter(np.sqrt(varianza_portafolioMaxRSCSShrunkCovariance), retorno_esperado_portMaxRSCSShrunkCovariance, color='b', marker='*', s=500)
# plt.ylim(0, 0.5)
# plt.xlim(0, 0.5)
plt.xlabel("Volatility")
plt.ylabel("Returns")